In [0]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [41]:
df = pd.read_csv("/content/drive/My Drive/Resid6_NeuralNetworks/Project/bank.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis =1 )

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Geography'] = le.fit_transform(df['Geography'])
df['Gender'] = le.fit_transform(df['Gender'])


In [0]:
x = df.iloc[:,0:10]
y = df.iloc[:,-1]

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, random_state = 9,stratify = y )

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_norm = sc.fit_transform(x_train)
x_test_norm = sc.transform(x_test)



In [120]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
lr = LogisticRegression()
lr.fit(x_train,y_train)
print(accuracy_score(y_train, lr.predict(x_train)), accuracy_score(y_test, lr.predict(x_test)))

0.7907142857142857 0.785


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [121]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
print(accuracy_score(y_train, knn.predict(x_train)), accuracy_score(y_test, knn.predict(x_test)))

0.8124285714285714 0.7636666666666667


In [122]:
# Naive Bayers
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
nb = GaussianNB()
nb.fit(x_train,y_train)
print(accuracy_score(y_train, nb.predict(x_train)), accuracy_score(y_test, nb.predict(x_test)))

0.7858571428571428 0.7853333333333333


In [123]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
dt = DecisionTreeClassifier(min_samples_split= 10, max_depth= 5)
dt.fit(x_train,y_train)
print(accuracy_score(y_train, dt.predict(x_train)), accuracy_score(y_test, dt.predict(x_test)))
print(confusion_matrix(y_test,dt.predict(x_test)))
print(classification_report(y_test,dt.predict(x_test)))

0.8575714285714285 0.8553333333333333
[[2345   44]
 [ 390  221]]
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      2389
           1       0.83      0.36      0.50       611

    accuracy                           0.86      3000
   macro avg       0.85      0.67      0.71      3000
weighted avg       0.85      0.86      0.83      3000



In [124]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
rf = RandomForestClassifier(n_estimators= 50, max_features = 'auto', bootstrap = True, max_depth = 5)
rf.fit(x_train,y_train)
print(accuracy_score(y_train, rf.predict(x_train)), accuracy_score(y_test, rf.predict(x_test)))
print(rf.feature_importances_)
x.columns


0.8594285714285714 0.857
[0.02002944 0.04342758 0.01023317 0.41821328 0.00760179 0.06369824
 0.32342697 0.00091372 0.09928502 0.01317078]


Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [125]:
# Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
rf = RandomForestClassifier(n_estimators= 50, max_features = 'auto', bootstrap = True, max_depth = 5)
bag = BaggingClassifier(n_estimators= 100,base_estimator= rf)
bag.fit(x_train,y_train)
print(accuracy_score(y_train, bag.predict(x_train)), accuracy_score(y_test, bag.predict(x_test)))


0.8582857142857143 0.855


In [0]:
from mlxtend.classifier import StackingClassifier

In [109]:
# Stacking Classifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
#estimators = [('rf',RandomForestClassifier(n_estimators=50)),('dt',DecisionTreeClassifier(max_depth=5))]
st = StackingClassifier(classifiers=[dt,rf,knn,bag], meta_classifier= lr)
st.fit(x_train,y_train)
print(accuracy_score(y_train, st.predict(x_train)), accuracy_score(y_test, st.predict(x_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8605714285714285 0.8553333333333333


In [110]:
# Boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
ab = AdaBoostClassifier()
ab.fit(x_train,y_train)
print(accuracy_score(y_train, ab.predict(x_train)), accuracy_score(y_test, ab.predict(x_test)))

0.8602857142857143 0.8533333333333334


In [113]:
# Stacking Classifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
#estimators = [('rf',RandomForestClassifier(n_estimators=50)),('dt',DecisionTreeClassifier(max_depth=5))]
st = StackingClassifier(classifiers=[dt,rf,knn,bag], meta_classifier= lr)
score = cross_val_score(st,x,y,cv =5,scoring = 'accuracy')
score
#print(accuracy_score(y_train, st.predict(x_train)), accuracy_score(y_test, st.predict(x_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

array([0.82108946, 0.86056972, 0.8145    , 0.82341171, 0.81190595])

In [114]:
score.mean()

0.8262953658488413

In [117]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [0]:
()